<a href="https://colab.research.google.com/github/vincentarlen/ChallengePlatinumBinar/blob/main/Nadia_Platinum_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd

df_train = pd.read_csv('/content/train.csv')
df_valid = pd.read_csv('/content/valid.csv')

df = df_train.append(df_valid, ignore_index=True)

df_test = pd.read_csv('/content/test.csv')

df = df.append(df_test, ignore_index=True)

<ipython-input-34-bd9281156118>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_train.append(df_valid, ignore_index=True)
<ipython-input-34-bd9281156118>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_test, ignore_index=True)


In [35]:
df.head()

,id,text,label
0,219,Nikmati cicilan 0% hingga 12 bulan untuk pemes...,neutral
1,209,Kue-kue yang disajikan bikin saya bernostalgia...,positive
2,436,Ibu pernah bekerja di grab indonesia,neutral
3,394,Paling suka banget makan siang di sini ayam sa...,positive
4,592,Pelayanan bus DAMRI sangat baik,positive


In [36]:
df.shape

(1000, 3)

In [37]:
df.label.value_counts()

negative    383
positive    378
neutral     239
Name: label, dtype: int64

In [68]:
import re
import pandas as pd

def fix_word(text):
    return ' '.join([kamus_dict[word] if word in kamus_dict else word for word in text.split(' ')])

def remove_unnecessaryChar(text):
     text = re.sub(r'&amp;|amp;|&', 'dan', text)
     text = re.sub(r'\\n+', '', text)
     text = re.sub('&lt;/?[a-z]+&gt;', ' ', text)
     text = re.sub(r'#+','#', text)
     text = re.sub(r'http\S+',' ',text)
     text = re.sub(r'(USER+\s?|RT+\s?|URL+\s?)', ' ', text)
     text = re.sub(r'x[a-zA-Z0-9]+', ' ', text)
     return text

def remove_punctuation(text):
     text = re.sub(r'\?', '', text)
     text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
     text = re.sub(r' +', ' ', text.lower().lstrip("0123456789").strip())
     return text

def preprocessing(text):
     text = remove_unnecessaryChar(text)
     text = remove_punctuation(text)
     text = fix_word(text)
     return text

df = pd.read_csv('train_preprocess.tsv.txt', encoding='ISO-8859-1', delimiter="\t", names=['text','label'])
df["text"] = df["text"].str.encode('ascii', 'ignore').str.decode('ascii')
df.drop_duplicates()
# print(df.head(30))
kamus = pd.read_csv('new_kamusalay.csv', names=['old','new'], encoding='ISO-8859-1')
kamus_dict = dict(zip(kamus['old'], kamus['new']))

df["text_clean"] = df["text"].apply(preprocessing)# apply cleansing
df.replace('', pd.NA, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df["text_clean"].head(30))
final = pd.DataFrame(df)
final.to_csv('try.csv')

df.head()

0     warung ini dimiliki oleh pengusaha pabrik tahu...
1     mohon ulama lurus dan k212 mmbri hujjah partai...
2     lokasi strategis di jalan sumatra bandung temp...
3     betapa bahagia nya diri ini saat unbo paket da...
4     aduh jadi mahasiswa jangan sombong dong kasih ...
5     makanan beragam harga makanan di food stall ak...
6     pakai kartu kredit baca tidak untung malah rug...
7     tempat unik bagus buat foto makanan enak pegaw...
8     saya bersama keluarga baru saja menikmati peng...
9                                             bersyukur
10    simcard indosat inaktiv gara gara lupa isi pul...
11    sifat iri sering muncul pada orang orang yang ...
12    sekadar menceritakan pengalaman saya pesan ste...
13    pengalaman bersama indosat hari ini semoga tid...
14                 anak sekarang sulit untuk dinasehati
15    hanya mengatasi masalah kayak begini saja anie...
16    setiap badan usaha milik negara dibentuk denga...
17    sehabis puas bermain di trans studio bandu

,text,label,text_clean
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mmbri hujjah partai...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatra bandung temp...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unbo paket da...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,aduh jadi mahasiswa jangan sombong dong kasih ...


In [69]:
data_preprocessed = df.text_clean.tolist()
data_preprocessed

['warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung tahu berkualitas dipadu keahlian memasak dipadu kretivitas jadilah warung yang menyajikan menu utama berbahan tahu ditambah menu umum lain seperti ayam semuanya selera indonesia harga cukup terjangkau jangan melewati tahu bletoka nya tidak kalah dengan yang asli dari tegal',
 'mohon ulama lurus dan k212 mmbri hujjah partai apa yang harus diwlh agar suara islam tidak pecah pecah',
 'lokasi strategis di jalan sumatra bandung tempat nya nyaman terutama sofa di lantai 2 paella nya enak sangat pas dimakan dengan minum bir dingin appetiser nya juga enak enak',
 'betapa bahagia nya diri ini saat unbo paket dan barang nya bagus menetapkan beli lagi',
 'aduh jadi mahasiswa jangan sombong dong kasih kartu kuning segala belajar dulu yang baik tidak usahlah ikut ikut politik nanti sudah selesai kuliah nya mau ikut politik juga tidak telat dasar mahasiswa',
 'makanan beragam harga makana

In [70]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print ("Feature Extraction selesai")

Feature Extraction selesai


In [71]:
import pickle

pickle.dump(count_vect, open("feature.p", "wb"))

In [66]:
#print(df.head().to_dict())

In [72]:
from sklearn.model_selection import train_test_split

classes = df.label

classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: label, Length: 11000, dtype: object

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

print ("Training selesai")

Training selesai


In [ ]:
pickle.dump(model, open("model.p", "wb"))

In [ ]:
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing selesai")

print(classification_report(y_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.76      0.72      0.74        76
     neutral       0.85      0.58      0.69        48
    positive       0.75      0.93      0.83        76

    accuracy                           0.77       200
   macro avg       0.79      0.75      0.76       200
weighted avg       0.78      0.77      0.76       200



In [74]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):
  data_train = X[data[0]]
  target_train = y[data[0]]

  data_test = X[data[1]]
  target_test = y[data[1]]

  clf = MultinomialNB()

  clf.fit(data_train,target_train)

  preds = clf.predict(data_test)

  accuracy = accuracy_score(target_test,preds)

  print("Training ke-", iteration)
  print(classification_report(target_test,preds))
  print("======================================")

  accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)


Training ke- 1
              precision    recall  f1-score   support

    negative       0.74      0.78      0.76       680
     neutral       0.89      0.69      0.78       239
    positive       0.88      0.89      0.88      1281

    accuracy                           0.83      2200
   macro avg       0.84      0.79      0.81      2200
weighted avg       0.84      0.83      0.83      2200

Training ke- 2
              precision    recall  f1-score   support

    negative       0.76      0.79      0.78       706
     neutral       0.84      0.64      0.72       220
    positive       0.88      0.90      0.89      1274

    accuracy                           0.84      2200
   macro avg       0.83      0.78      0.80      2200
weighted avg       0.84      0.84      0.84      2200

Training ke- 3
              precision    recall  f1-score   support

    negative       0.76      0.79      0.77       682
     neutral       0.90      0.68      0.78       215
    positive       0.89      0

In [77]:
original_text = '''
tidak sopan banget.
'''
text = count_vect.transform([preprocessing(original_text)])

result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative
